In [2]:
import pyaerocom as pya
from pyaerocom.io.read_emep import ReadEMEP
from pyaerocom.variable import get_emep_variables
import glob
import os
import xarray as xr

In [3]:
# Mapping of ts_type to EMEP filenames
ts_type_EMEP_filename = {'daily':'Base_day.nc', 'monthly':'Base_month.nc', 'yearly': 'Base_fullrun.nc'}

# Map of EMEP variable prefix to vertical type used for storing nc files
map_prefix_aero = {'AOD': 'Column' ,'AAOD' : 'Column', 'Abs_coeff': 'ModelLevel',
                   'AbsCoeff': 'ModelLevel',
                   'DDEP': 'Surface', 'Emis': 'Surface',
                   'COLUMN': 'Column', 'D3': 'ModelLevel',
                   'SURF': 'Surface', 'WDEP': 'Surface',
                  'z3d': 'Surface',
                  'Z': 'Surface'}



def EMEP_to_aerocom(in_folder, out_folder, ts_type, year, data_id, filename=None):
    """
    Converts all (known) variables from in_folder/filename to Aerocom format in out_folder.
    """
    
    if filename == None:
        try:
            filename = ts_type_EMEP_filename[ts_type]
        except KeyError as e:
            print('ts_type not recognized')
            #exit
    
    filepath = os.path.join(in_folder, filename)

    
    if not (os.path.isdir(in_folder) and os.path.isdir(out_folder)):
        raise FileNotFoundError('in_folder or out_folder is not a folder')
        # exit ?
    if not os.path.isfile(filepath):
        raise FileNotFoundError('File to be converted not found: {}'.format(filepath))
        # exit?
        
    reader = ReadEMEP(filepath, data_id=data_id)
    for key, var in get_emep_variables().items():
#         print('Converting {} / {}'.format(key, var))
        # Load data
        try:
            data = reader.read_var(key, ts_type=ts_type)
        except KeyError as e:
#             print('{} not found in {} EMEP files\n'.format(key, ts_type))
            continue
        except ValueError as e:
            print(repr(e))
            continue
        data.change_base_year(year) # Change year to match emission year
        
        # Infer vertical coordinate from variable naming
        type_emep = (var.split('_')[0])
        vert_code = map_prefix_aero[type_emep]
        data.to_netcdf(out_folder, vert_code=vert_code);


In [ ]:
basepath = '/home/eirikg/Desktop/pyaerocom/data/2020_AerocomHIST/'
store = '/home/eirikg/Desktop/pyaerocom/data/2020_AerocomHIST/processed'
data_id = 'EMEP-met2010'

years = [1850, 1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1985, 1990, 1995, 2000, 2005, 2010]
paths = [os.path.join(basepath, '{}_GLOB1_2010met'.format(year)) for year in years]

for ts_type in ['daily', 'monthly']:
    for (path, year) in  zip(paths, years):
        EMEP_to_aerocom(path, store, ts_type, year, data_id)

Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData contains only 1850
Nothing to split... GriddedData